In [1]:
import pandas as pd, numpy as np
import tensorflow as tf
from tensorflow import keras

In [4]:
train = pd.read_csv('/home/thompson/Downloads/train.csv')
test = pd.read_csv('/home/thompson/Downloads/test.csv')

# Data Exploration

In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [7]:
text = train['comment_text']

In [8]:
text[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [9]:
train['comment_text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [10]:
# for train
lens = train.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

(394.0732213246768, 590.7202819048923, 5000)

In [11]:
# for test
lens = test.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

(364.8751207855632, 592.4920987303017, 5000)

In [12]:
lens.hist()

In [13]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1) ## each column may have the value of one ( Labled ) . 1- calc the max # if has no lable max = 0 then col = 1 -0 = 0
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805,0.898321
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420,0.302226
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
len(train),len(test)

(159571, 153164)

In [15]:
## deal with nulls 
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [18]:
import re, string

In [19]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): 
    return re_tok.sub(r' \1 ', s).split()

In [20]:
def clean(s): 
    return re_tok.sub(r' \1 ', s)

In [21]:
# vocab size 
words = []
for t in text:
    words.extend(tokenize(t))
print(words[:100])
vocab = list(set(words))
print(len(words), len(vocab))

['Explanation', 'Why', 'the', 'edits', 'made', 'under', 'my', 'username', 'Hardcore', 'Metallica', 'Fan', 'were', 'reverted', '?', 'They', 'weren', "'", 't', 'vandalisms', ',', 'just', 'closure', 'on', 'some', 'GAs', 'after', 'I', 'voted', 'at', 'New', 'York', 'Dolls', 'FAC', '.', 'And', 'please', 'don', "'", 't', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'I', "'", 'm', 'retired', 'now', '.', '89', '.', '205', '.', '38', '.', '27', 'D', "'", 'aww', '!', 'He', 'matches', 'this', 'background', 'colour', 'I', "'", 'm', 'seemingly', 'stuck', 'with', '.', 'Thanks', '.', '(', 'talk', ')', '21', ':', '51', ',', 'January', '11', ',', '2016', '(', 'UTC', ')', 'Hey', 'man', ',', 'I', "'", 'm', 'really', 'not']
13610355 234714


In [22]:
train['comment_text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [23]:
clean(train['comment_text'][0])

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted ?  They weren ' t vandalisms ,  just closure on some GAs after I voted at New York Dolls FAC .  And please don ' t remove the template from the talk page since I ' m retired now . 89 . 205 . 38 . 27"

In [24]:
def one_hot_word_embedding(vtrain_data, vtest_data):
    # switch data back to text 
    train_labels = vtrain_data[label_cols]
    txt_train_data = [clean(txt) for txt in train['comment_text']]
    txt_test_data = [clean(txt) for txt in test['comment_text']]
    
    # integer encode the documents
    vocab_size = 10000
    encoded_txt_train_data = [keras.preprocessing.text.one_hot(d, vocab_size) for d in txt_train_data]
    encoded_txt_test_data = [keras.preprocessing.text.one_hot(d, vocab_size) for d in txt_test_data]
    #print(encoded_txt_train_data)

    ptxt_train_data = keras.preprocessing.sequence.pad_sequences(encoded_txt_train_data,
                                                            padding='post',
                                                            maxlen=5000)

    ptxt_test_data = keras.preprocessing.sequence.pad_sequences(encoded_txt_test_data,
                                                           padding='post',
                                                           maxlen=5000)
    x_val = ptxt_train_data[:100000] 
    partial_x_train = ptxt_train_data[100000:]

    y_val = train_labels[:100000]
    partial_y_train = train_labels[100000:]
    return (x_val,partial_x_train,y_val,partial_y_train,ptxt_test_data)


In [25]:

def full_one_hot_word_embedding(vtrain_data,vtest_data):
    # switch data back to text 
    train_labels = vtrain_data[label_cols]
    txt_train_data = [clean(txt) for txt in train['comment_text']]
    txt_test_data = [clean(txt) for txt in test['comment_text']]
    
    # integer encode the documents
    vocab_size = 10000
    encoded_txt_train_data = [keras.preprocessing.text.one_hot(d, vocab_size) for d in txt_train_data]
    encoded_txt_test_data = [keras.preprocessing.text.one_hot(d, vocab_size) for d in txt_test_data]
    #print(encoded_txt_train_data)

    ptxt_train_data = keras.preprocessing.sequence.pad_sequences(encoded_txt_train_data,
                                                            padding='post',
                                                            maxlen=5000)

    ptxt_test_data = keras.preprocessing.sequence.pad_sequences(encoded_txt_test_data,
                                                           padding='post',
                                                           maxlen=5000)
    partial_x_train = ptxt_train_data
    partial_y_train = train_labels
    return (partial_x_train,partial_y_train,ptxt_test_data)

In [26]:
# def model_with_emb_acc(vtrain_data,vtest_data,vocab_size = 10000):
#     model1 = keras.Sequential()
#     model1.add(keras.layers.Embedding(vocab_size, 16))
#     model1.add(keras.layers.GlobalAveragePooling1D())
#     model1.add(keras.layers.Dense(512, activation=tf.nn.relu))
#     model1.add(keras.layers.Dense(6, activation=tf.nn.sigmoid))
#     model1.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['acc'])
#     x_val,partial_x_train,y_val,partial_y_train,test_data = one_hot_word_embedding(vtrain_data,vtest_data)
#     earlystopper = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
#     history = model1.fit(x_val,
#                      y_val,
#                      epochs=20,
#                      callbacks=[earlystopper],
#                      batch_size=512,
#                      validation_data=(x_val, y_val),
#                      verbose=1)
# #   results1 = model1.evaluate(x_val, y_val)
#     return (model1,test_data,history)

In [27]:
def full_model_with_emb_acc(vtrain_data,vtest_data,vocab_size = 10000):
    model1 = keras.Sequential()
    model1.add(keras.layers.Embedding(vocab_size, 16))
    model1.add(keras.layers.GlobalAveragePooling1D())
    model1.add(keras.layers.Dense(512, activation=tf.nn.relu))
    model1.add(keras.layers.Dense(6, activation=tf.nn.sigmoid))
    model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
    partial_x_train,partial_y_train,test_data = full_one_hot_word_embedding(vtrain_data,vtest_data)
    earlystopper = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    history = model1.fit(partial_x_train,
                     partial_y_train,
                     epochs=20,
                     callbacks=[earlystopper],
                     batch_size=1024,
                     verbose=1)
#   results1 = model1.evaluate(x_val, y_val)
    return (model1, test_data, history)

# Train / Test

In [28]:
model1, test_data, his1 = full_model_with_emb_acc(train,test)

Epoch 1/20
159571/159571 [==============================] - 82s 515us/sample - loss: 0.2596 - acc: 0.9603
Epoch 2/20
159571/159571 [==============================] - 81s 508us/sample - loss: 0.1410 - acc: 0.9633
Epoch 3/20
159571/159571 [==============================] - 81s 507us/sample - loss: 0.1407 - acc: 0.9633
Epoch 4/20
159571/159571 [==============================] - 81s 509us/sample - loss: 0.1404 - acc: 0.9633
Epoch 5/20
159571/159571 [==============================] - 81s 511us/sample - loss: 0.1401 - acc: 0.9633
Epoch 6/20
159571/159571 [==============================] - 82s 514us/sample - loss: 0.1396 - acc: 0.9633
Epoch 7/20
159571/159571 [==============================] - 81s 510us/sample - loss: 0.1389 - acc: 0.9634
Epoch 8/20
159571/159571 [==============================] - 84s 524us/sample - loss: 0.1371 - acc: 0.9634
Epoch 9/20
159571/159571 [==============================] - 84s 528us/sample - loss: 0.1340 - acc: 0.9635
Epoch 10/20
159571/159571 [===================

In [41]:
y_hat = model1.predict(test_data, batch_size=1024)

In [42]:
y_hat

array([[9.7724909e-01, 2.2120598e-01, 7.8460228e-01, 4.3894947e-02,
        7.8896004e-01, 1.4230275e-01],
       [5.2026182e-02, 5.5297911e-03, 2.4910301e-02, 2.6593804e-03,
        2.2120684e-02, 6.3656569e-03],
       [7.9448223e-02, 7.5793564e-03, 3.7755847e-02, 3.3310652e-03,
        3.3986747e-02, 8.2931519e-03],
       ...,
       [8.0744326e-03, 1.1092722e-03, 3.8770139e-03, 7.6392293e-04,
        3.0883253e-03, 1.6106963e-03],
       [1.2584597e-02, 1.9344389e-03, 6.3088238e-03, 1.2625456e-03,
        5.3074360e-03, 2.6450753e-03],
       [2.2500393e-01, 1.7848134e-02, 1.1071947e-01, 6.2272251e-03,
        1.0336909e-01, 1.7065704e-02]], dtype=float32)